<a href="https://colab.research.google.com/github/hllj/fer-dexpression/blob/master/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

!pip install -q -U opencv-python

In [0]:
root_path = 'gdrive/My Drive/FER-Model' #my folder directory 

In [0]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import glob
import cv2
from __future__ import print_function

## Get all images from CK+ dataset:

1.   Anger
2.   Contempt
3.   Disgust
4. Fear
5. Happiness
6. Neural
7. Sadness
8. Suprise



In [0]:
frames = []
list_emotions = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']


for idx, emotion in enumerate(list_emotions):
  list_folder_images = glob.glob(root_path + '/CK+/' + emotion + '/*')
  print("Number of {} images: {}".format(emotion, len(list_folder_images)))
  
  emotion_folder = [root_path + '/CK+/' + emotion + '/' for i in list_folder_images]
  emotion_imagename = [i.split('/')[-1] for i in list_folder_images]
  emotion_name = [[emotion]*len(list_folder_images)][0]
  emotion_label = [idx]*len(list_folder_images)

  print(len(emotion_folder), len(emotion_imagename), len(emotion_name), len(emotion_label))
  
  df_emotion = pd.DataFrame()
  df_emotion['Folder'] = emotion_folder
  df_emotion['ImageName'] = emotion_imagename
  df_emotion['Emotion'] = emotion_name
  df_emotion['Label'] = emotion_label
  frames.append(df_emotion)
  

Number of anger images: 45
45 45 45 45
Number of contempt images: 18
18 18 18 18
Number of disgust images: 59
59 59 59 59
Number of fear images: 25
25 25 25 25
Number of happiness images: 69
69 69 69 69
Number of neutral images: 593
593 593 593 593
Number of sadness images: 28
28 28 28 28
Number of surprise images: 82
82 82 82 82


In [0]:
final_df = pd.concat(frames)
final_df

,Folder,ImageName,Emotion,Label
0,gdrive/My Drive/FER-Model/CK+/anger/,S075_008_00000012.png,anger,0
1,gdrive/My Drive/FER-Model/CK+/anger/,S066_005_00000011.png,anger,0
2,gdrive/My Drive/FER-Model/CK+/anger/,S067_004_00000023.png,anger,0
3,gdrive/My Drive/FER-Model/CK+/anger/,S071_004_00000028.png,anger,0
4,gdrive/My Drive/FER-Model/CK+/anger/,S072_005_00000019.png,anger,0
...,...,...,...,...
77,gdrive/My Drive/FER-Model/CK+/surprise/,S011_001_00000016.png,surprise,7
78,gdrive/My Drive/FER-Model/CK+/surprise/,S130_001_00000018.png,surprise,7
79,gdrive/My Drive/FER-Model/CK+/surprise/,S131_001_00000016.png,surprise,7
80,gdrive/My Drive/FER-Model/CK+/surprise/,S136_001_00000019.png,surprise,7


In [0]:
final_df.reset_index(inplace = True, drop = True)
final_df = final_df.sample(frac = 1.0)
final_df.head()

,Folder,ImageName,Emotion,Label
127,gdrive/My Drive/FER-Model/CK+/fear/,S124_003_00000011.png,fear,3
792,gdrive/My Drive/FER-Model/CK+/neutral/,S137_001_00000001.png,neutral,5
190,gdrive/My Drive/FER-Model/CK+/happiness/,S065_004_00000028.png,happiness,4
56,gdrive/My Drive/FER-Model/CK+/contempt/,S154_002_00000013.png,contempt,1
488,gdrive/My Drive/FER-Model/CK+/neutral/,S077_005_00000001.png,neutral,5


## Train, Test split

In [0]:
from sklearn.model_selection import train_test_split


In [0]:
df_train, df_test = train_test_split(final_df, stratify=final_df['Label'], test_size=0.1)

In [0]:
print('Train size: ', df_train.shape)
print('Test size: ', df_test.shape)

Train size:  (827, 4)
Test size:  (92, 4)


In [0]:
df_train.reset_index(inplace = True, drop = True)
df_test.reset_index(inplace = True, drop = True)

## Pre-Processing Images


### 1.   Converting all images to grayscale and save them

In [0]:
def convt_to_gray(df):
  count = 0
  for i in range(len(df)):
    path1 = df["Folder"][i]
    path2 = df["ImageName"][i]
    img = cv2.imread(path1 + path2)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imwrite(path1 + path2, gray)
    count += 1
  print("Total number of images converted and saved = "+str(count))

In [0]:
convt_to_gray(df_train)

Total number of images converted and saved = 827


In [0]:
convt_to_gray(df_test)

Total number of images converted and saved = 92


In [0]:
def resize_image(df):
  count = 0
  for i in range(len(df)):
    path1 = df["Folder"][i]
    path2 = df["ImageName"][i]
    img = cv2.imread(path1 + path2)
    cv2.imwrite(path1 + path2, cv2.resize(img, (350, 350)))
    count += 1
  print("Total number of images resized and saved = "+str(count))

In [0]:
resize_image(df_train)

Total number of images resized and saved = 827


In [0]:
resize_image(df_test)

Total number of images resized and saved = 92


### 2.Detecting face in image using HAAR then crop it then resize then save the image

In [0]:
# #detect the face in image using HAAR cascade then crop it then resize it and finally save it.
# face_cascade = cv2.CascadeClassifier(root_path + '/haarcascade_frontalface_default.xml') 
# #download this xml file from link: https://github.com/opencv/opencv/tree/master/data/haarcascades.
# def face_det_crop_resize(img_path):
#     img = cv2.imread(img_path)
#     gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#     flag = False
#     for (x,y,w,h) in faces:
#         flag = True #success to crop
#         face_clip = img[y:y+h, x:x+w]  #cropping the face in image
#         cv2.imwrite(img_path, cv2.resize(face_clip, (224, 244)))  #resizing image then saving it
#     return flag

In [0]:

# for i, d in df_train.iterrows():
#     img_path = d['Folder'] + d['ImageName']
#     face_det_crop_resize(img_path)

In [0]:

# for i, d in df_test.iterrows():
#     img_path = d['Folder'] + d['ImageName']
#     flag = face_det_crop_resize(img_path)
#     if (flag == False):
#       print("Failed at image ", i)

In [0]:
df_train.to_csv(r'gdrive/My Drive/FER-Model/train.csv', index=None, header=True)
df_test.to_csv(r'gdrive/My Drive/FER-Model/test.csv', index=None, header=True)